In [1]:
import config
from utils import get_chromedriver, get_seduta, get_studenti, export_triennali


import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
from bs4 import BeautifulSoup
from soup2dict import convert
import pandas as pd
import numpy as np
import xlsxwriter
import os
import joblib
import locale



locale.setlocale(locale.LC_ALL, 'it_IT')

'it_IT'

In [2]:
CHROMEDRIVER_PATH = r"C:\Users\Alessandro Bitetto\Downloads\UniPV\Tool Sedute Laurea\WebDriver\chromedriver"

In [3]:
GRADUATION_DATE = ['26/02/2024', '27/02/2024']    # keep list also with 1 value
ESSE3_URL = "https://studentionline.unipv.it/esse3/Home.do"
ROOT_URL = "https://studentionline.unipv.it/"    # used to open link inside esse3
# triennali
TRIENNALI = True
QUERY_TRIENNALI = 'Query laurendi TRIENNALI FEBBRAIO con medie DEF.xlsxP.xlsx'   # in folder Query
TEMPESTIVITA_YEAR = '2021/2022'
MAPPING_CORSO=pd.DataFrame({'Corso': ['MANAGEMENT', 'ECONOMIA', 'AMMINISTRAZIONE, CONTROLLO E FINANZA AZIENDALE'],
                            'Corso_lab': ['Corso di laurea in Management classe L-18', 'Corso di laurea in Economia, classe L-33',
                            'Corso di Laurea in Amministrazione, controllo e finanza aziendale classe L-18']})
# magistrali
CHECK_YEAR = '2013/2014'    # if anno iscrizione is before CHECK_YEAR, corresponding cell is filled in yellow

# Download Lista Studenti

In [4]:
# get seduta session
driver, link_list=get_seduta(CHROMEDRIVER_PATH, GRADUATION_DATE, ESSE3_URL, ROOT_URL, TRIENNALI, config)

df_studenti=get_studenti(driver, link_list, MAPPING_CORSO, GRADUATION_DATE)



########################### 26/02/2024 ore 09:00  (Palazzo San Felice - Sala lettura Chiesa) ###########################

- Found 16 in commissione: 


,Docente,Ruolo
1,ZUCCHELLA ANTONELLA,Supplente
2,MACCARINI MAURIZIO ETTORE,Supplente
3,ASCARI GUIDO,Supplente
4,SASSI MARIA,Supplente
5,BERETTA VALENTINA,Supplente
6,MORIANI SERGIO,Supplente
7,TANDA ALESSANDRA,Supplente
8,VERONA BRUNO WALTER,Supplente
9,GOTTARDO PIETRO,Supplente
10,GUAITA MARCO,Supplente



File saved in .\Sedute\Log\Seduta Laurea 2024_02_26_09.00_commissione.csv

- Found 2 pages

- Found 12 students
Reading student 12/12

########################### 26/02/2024 ore 14:00  (Palazzo San Felice - Sala lettura Chiesa) ###########################

- Found 18 in commissione: 


,Docente,Ruolo
1,CAVALIERE ALBERTO,Supplente
2,CORRADI FIAMMETTA,Supplente
3,FUMAGALLI ANDREA MARIA PIERO CAMILLO,Supplente
4,SPELTA ALESSANDRO,Supplente
5,GRAZZINI JAKOB,Supplente
6,RE BEATRICE,Supplente
7,GERACI ANDREA,Supplente
8,GOTTARDO PIETRO,Supplente
9,NAVARONI MATTEO,Supplente
10,LAZZARI DANIELA,Supplente



File saved in .\Sedute\Log\Seduta Laurea 2024_02_26_14.00_commissione.csv

- Found 10 students
Reading student 10/10

########################### 27/02/2024 ore 09:00  (Palazzo San Felice - Sala lettura Chiesa) ###########################

- Found 16 in commissione: 


...

,Docente,Ruolo
1,VERONA BRUNO WALTER,Supplente
2,CORRADI FIAMMETTA,Supplente
3,PELLICELLI MICHELA,Supplente
4,SASSI MARIA,Supplente
5,RIZZO MARIO VALENTINO,Supplente
6,PELLATI GIORGIO,Supplente
7,GOTTARDO PIETRO,Supplente
8,BALLARANI GIANLUIGI,Supplente
9,GORGONI STEFANO,Supplente
10,FAGGIOLI GABRIELE,Supplente



File saved in .\Sedute\Log\Seduta Laurea 2024_02_27_14.00_commissione.csv

- Found 10 students


,Data,Luogo,Nome,Matricola,Nascita,Corso,Titolo tesi,Relatore,Anno iscrizione,Crediti,...,Motivazione punteggio proposto,Richiesta di eccellenza,Punti tesi,Lode,Encomio,Voto Finale,link,link_seduta,file_commissione,Corso_lab
0,26/02/2024 ore 09:00,Palazzo San Felice - Sala lettura Chiesa,IPPOLITO CARLOS,490786,30/11/2001,MANAGEMENT,Analisi e valutazione degli investimenti finan...,VERONA BRUNO WALTER,1 2020/2021 In corso\n2 2021/2022 In corso\n3 ...,177,...,,No,,,,0,auth/docente/Graduation/DettaglioLaureando.do?...,https://studentionline.unipv.it/auth/docente/G...,.\Sedute\Log\Seduta Laurea 2024_02_26_09.00_co...,Corso di laurea in Management classe L-18
1,26/02/2024 ore 09:00,Palazzo San Felice - Sala lettura Chiesa,BELOTTI SIMONE,488915,04/11/2001,MANAGEMENT,L'utilizzo dell'AI e dei Big data per le scelt...,TANDA ALESSANDRA,1 2020/2021 In corso\n2 2021/2022 In corso\n3 ...,177,...,,No,,,,0,auth/docente/Graduation/DettaglioLaureando.do?...,https://studentionline.unipv.it/auth/docente/G...,.\Sedute\Log\Seduta Laurea 2024_02_26_09.00_co...,Corso di laurea in Management classe L-18
2,26/02/2024 ore 09:00,Palazzo San Felice - Sala lettura Chiesa,CASSANI EDOARDO,478365,02/02/1999,MANAGEMENT,Il potenziale economico e commerciale dell'Int...,MORIANI SERGIO,1 2019/2020 In corso\n2 2020/2021 In corso\n3 ...,177,...,,No,,,,0,auth/docente/Graduation/DettaglioLaureando.do?...,https://studentionline.unipv.it/auth/docente/G...,.\Sedute\Log\Seduta Laurea 2024_02_26_09.00_co...,Corso di laurea in Management classe L-18
3,26/02/2024 ore 09:00,Palazzo San Felice - Sala lettura Chiesa,MANZONI RICCARDO JAMES,490432,05/05/2001,MANAGEMENT,UTILIZZO DELLE INTELLIGENZE ARTIFICIALI NEL SE...,BERETTA VALENTINA,1 2020/2021 In corso\n2 2021/2022 In corso\n3 ...,177,...,,No,,,,0,auth/docente/Graduation/DettaglioLaureando.do?...,https://studentionline.unipv.it/auth/docente/G...,.\Sedute\Log\Seduta Laurea 2024_02_26_09.00_co...,Corso di laurea in Management classe L-18
4,26/02/2024 ore 09:00,Palazzo San Felice - Sala lettura Chiesa,PRIFTI ARLISA,488928,15/12/2000,MANAGEMENT,La Sostenibilità all'interno delle Imprese: il...,BERETTA VALENTINA,1 2020/2021 In corso\n2 2021/2022 In corso\n3 ...,177,...,,No,,,,0,auth/docente/Graduation/DettaglioLaureando.do?...,https://studentionline.unipv.it/auth/docente/G...,.\Sedute\Log\Seduta Laurea 2024_02_26_09.00_co...,Corso di laurea in Management classe L-18


0
Data                 Corso_lab                                             
26/02/2024 ore 09:00 Corso di laurea in Management classe L-18           12
26/02/2024 ore 14:00 Corso di laurea in Economia, classe L-33            10
27/02/2024 ore 09:00 Corso di laurea in Economia, classe L-33             6
                     Corso di laurea in Management classe L-18            4
27/02/2024 ore 14:00 Corso di Laurea in Amministrazione, controllo e...  10


Log studenti saved in Checkpoints\Seduta Laurea 2024_02_26,27 - df_studenti.pkl


## Export report TRIENNALI

In [4]:
# File to be sent for checking Punteggio Tesi and Voto

file_name_pkl=os.path.join('Checkpoints', 'Seduta Laurea '+'_'.join(GRADUATION_DATE[0].split('/')[::-1])+','+','.join([x[:2] for x in GRADUATION_DATE[1:]])+' - df_studenti.pkl')
df_studenti=joblib.load(file_name_pkl)

export_triennali(df_studenti, QUERY_TRIENNALI, GRADUATION_DATE, TEMPESTIVITA_YEAR, final_version=False)

#### found 2 rows with mismatch in Anno iscrizione


,Data,Luogo,Nome,Matricola,Nascita,Corso,Titolo tesi,Relatore,Anno iscrizione,Crediti,...,link_seduta,file_commissione,Corso_lab,NOME,COGNOME,MATRICOLA,PUNTI_TOTALI,AA_IMM_SU,check voto,check anno iscrizione
5,26/02/2024 ore 09:00,Palazzo San Felice - Sala lettura Chiesa,ONOFRI ALESSANDRA,502114,28/10/2000,MANAGEMENT,La filiera verticalmente integrata come strate...,SASSI MARIA,1 2020/2021 In corso\n2 2021/2022 In corso\n3 ...,177,...,https://studentionline.unipv.it/auth/docente/G...,.\Sedute\Log\Seduta Laurea 2024_02_26_09.00_co...,Corso di laurea in Management classe L-18,ALESSANDRA,ONOFRI,502114,81.000,2019,True,False
15,26/02/2024 ore 14:00,Palazzo San Felice - Sala lettura Chiesa,SCARCELLA DAVIDE,493602,31/12/2000,ECONOMIA,La BCE e gli effetti sociali delle politiche m...,GRAZZINI JAKOB,1 2020/2021 In corso\n2 2021/2022 In corso\n3 ...,177,...,https://studentionline.unipv.it/auth/docente/G...,.\Sedute\Log\Seduta Laurea 2024_02_26_14.00_co...,"Corso di laurea in Economia, classe L-33",DAVIDE,SCARCELLA,493602,90.933,2019,True,False


,0
Tempestività,
No,42



Studenti list saved in Sedute\Log\Seduta Laurea 2024_02_26,27_studenti.csv

File saved in Sedute\Seduta Laurea 2024_02_26,27.xlsx

Email list saved in Sedute\Seduta Laurea 2024_02_26,27_email.csv


In [4]:
# Final version to be sent to Proclamatori

file_name_pkl=os.path.join('Checkpoints', 'Seduta Laurea '+'_'.join(GRADUATION_DATE[0].split('/')[::-1])+','+','.join([x[:2] for x in GRADUATION_DATE[1:]])+' - df_studenti.pkl')
df_studenti=joblib.load(file_name_pkl)

export_triennali(df_studenti, QUERY_TRIENNALI, GRADUATION_DATE, TEMPESTIVITA_YEAR, final_version=True)

File saved in Sedute\Seduta Laurea 2024_02_26,27 BAGNA.xlsx
File saved in Sedute\Seduta Laurea 2024_02_26,27 GENNUSA.xlsx
File saved in Sedute\Seduta Laurea 2024_02_26,27 CAVALIERE.xlsx
File saved in Sedute\Seduta Laurea 2024_02_26,27 ASCARI.xlsx


## Export report MAGISTRALI

In [81]:
file_name='.\\Sedute\\Seduta Laurea '+'_'.join(GRADUATION_DATE[0].split('/')[::-1])+'.xlsx'
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df_studenti.to_excel(writer, sheet_name='Laurea', index=False)

# Get the xlsxwriter objects from the dataframe writer object.
workbook  = writer.book
worksheet = writer.sheets['Laurea']

wrap_format = workbook.add_format({'text_wrap' : True, 'align': 'top'})
top_format=workbook.add_format({'align': 'top'})
yellow_format=workbook.add_format({'text_wrap' : True, 'align': 'top', 'bg_color': 'yellow'})
worksheet.set_column(0, 5, 15, top_format)
worksheet.set_column(7, 9, 15, top_format)
worksheet.set_column(11, 11, 20, top_format)
worksheet.set_column(6, 6, 20, wrap_format)
worksheet.set_column(10, 10, 20, wrap_format)
worksheet.set_column(12, 12, 30, wrap_format)

# add formula for voto finale
col_num=df_studenti.columns.get_loc("Voto Finale")+1
col_letter_finale=chr(ord('@')+col_num)
col_num=df_studenti.columns.get_loc("Voto proposto")+1
col_letter_proposto=chr(ord('@')+col_num)
col_num=df_studenti.columns.get_loc("Punti tesi")+1
col_letter_punti=chr(ord('@')+col_num)
for ind, row in df_studenti.iterrows():
    worksheet.write_formula(col_letter_finale+str(ind+2), '='+ col_letter_proposto+str(ind+2) +' + '+ col_letter_punti+str(ind+2))
    # https://xlsxwriter.readthedocs.io/working_with_formulas.html
    
# fill cell with yellow if "Anno iscrizione" is before CHECK_YEAR
col_num=df_studenti.columns.get_loc("Anno iscrizione")+1
col_letter=chr(ord('@')+col_num)
for ind, row in df_studenti.iterrows():
    anno=[]
    for an in row['Anno iscrizione'].split('\n'):
        anno.append([int(x.split('/')[0]) for i, x in enumerate(an.split(' ')) if '/' in x][0])
    if min(anno) <= int(CHECK_YEAR.split('/')[0]):
        worksheet.write(col_letter+str(ind+2), row['Anno iscrizione'], yellow_format)

writer.close()

print('\nFile saved in', file_name)


File saved in Seduta Laurea 03_10_2023.xlsx


C:\Users\Alessandro Bitetto\AppData\Local\Temp\ipykernel_17556\1437380815.py:42: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


# Upload Voti



Prima compila la colonna `Punti tesi` e `Lode` nel file excel

In [4]:
# df_commissione=pd.read_csv('Seduta Laurea '+'_'.join(GRADUATION_DATE.split('/')[::-1])+'_commissione.csv', sep=';')
df_studenti=pd.read_excel('.\\Sedute\\Seduta Laurea '+'_'.join(GRADUATION_DATE[0].split('/')[::-1])+'.xlsx')
df_studenti['Lode']=df_studenti['Lode'].fillna('')
df_studenti['Encomio']=df_studenti['Encomio'].fillna('')

In [32]:
# get seduta session
driver=get_seduta(CHROMEDRIVER_PATH, GRADUATION_DATE, ESSE3_URL, ROOT_URL, config)
wait=WebDriverWait(driver, 10)

In [6]:
driver.switch_to.window(driver.window_handles[0])

for i, row in df_studenti.iterrows():
    
    print(f"Uploading student {i+1}/{len(df_studenti)}", end = "\r")
    
    link=ROOT_URL+row['link']
    driver.execute_script(f"window.open('{link}')")
    new_tab= driver.window_handles[i+1]
    driver.switch_to.window(new_tab)
    
    # input punti tesi
    try:
        punti_tesi=wait.until(
                        EC.presence_of_element_located((By.ID, 'grad-dettLau-puntiTesi')))
        punti_tesi.click()
        punti_tesi.clear()
        punti_tesi.send_keys(row['Punti tesi'])
        wait.until(EC.presence_of_element_located((By.ID, 'grad-dettLau-annotazioni'))).click()
    except:
        print(f'\n## Error for punti tesi {row["Nome"]} ({i+1})')

    # tick Lode and Encomio
    if row['Lode'] != '':
        try:
            lode=wait.until(
                    EC.presence_of_element_located((By.ID, 'grad-dettLau-lode1')))
            lode.click()
        except:
            print(f'\n## Error for Lode {row["Nome"]} ({i+1})')
    if row['Encomio'] != '':
        try:
            encomio=wait.until(
                    EC.presence_of_element_located((By.ID, 'grad-dettLau-encomio1')))
            encomio.click()
        except:
            print(f'\n## Error for Encomio {row["Nome"]} ({i+1})')

    # tick Commissione
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        commissione=wait.until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div/div/main/div[3]/div/div/div[2]/form/div[3]")))
        commissione=convert(BeautifulSoup(commissione.get_attribute('outerHTML'), 'html.parser'))
        get_checkbox_id=commissione['div'][0]['div'][0]['@id'].replace('gbox', 'cb')
        all_commissione=wait.until(
                        EC.presence_of_element_located((By.ID, get_checkbox_id)))
        all_commissione.click()
    except:
        print(f'\n## Error for Commissione {row["Nome"]} ({i+1})')

    # save
    try:
        save=wait.until(
                    EC.presence_of_element_located((By.ID, 'grad-dettLau-btnSubmit')))
        save.click()
    except:
        print(f'\n## Error for Save {row["Nome"]} ({i+1})')

    save and exit
    try:
        save_exit=wait.until(
                    EC.presence_of_element_located((By.ID, 'grad-dettLau-btnSubmitExit')))
        save_exit.click()
    except:
        print(f'\n## Error for Save and Exit {row["Nome"]} ({i+1})')


# check final grade mismatch
driver.switch_to.window(driver.window_handles[0])

if "footable-page-link" not in driver.page_source:     # single page
    final_grade=wait.until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div/div/main/div[3]/div/div/div[2]/table[2]")))
    final_grade=convert(BeautifulSoup(final_grade.get_attribute('outerHTML'), 'html.parser'))
    elem_list=final_grade['table'][0]['tbody'][0]['tr']
else:                                                  # multiple pages
    expected_rows = int(driver.find_element(By.XPATH, "/html/body/div[2]/div/div/main/div[3]/div/div/div[2]/table[2]/tfoot/tr/td/div/span").get_attribute("textContent").split(' di ')[-1])
    num_pages = int(driver.find_element(By.XPATH, "/html/body/div[2]/div/div/main/div[3]/div/div/div[2]/table[2]/tfoot/tr/td/div/span").get_attribute("textContent").split(',')[0].split('di')[-1])
    
    print(f'\n- Found {num_pages} pages')
        
    elem_list=[]
    for i in range(num_pages):
        
        time.sleep(1)
        final_grade=wait.until(
            EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div/div/main/div[3]/div/div/div[2]/table[2]")))
        final_grade=convert(BeautifulSoup(final_grade.get_attribute('outerHTML'), 'html.parser'))
        elem_list.extend(final_grade['table'][0]['tbody'][0]['tr'])

        if i < (num_pages-1):
            driver.find_element(By.XPATH, "/html/body/div[2]/div/div/main/div[3]/div/div/div[2]/table[2]/tfoot/tr/td/div/ul/li[5]/a").click()
            time.sleep(2)
                
    print(f'- Found {len(elem_list)} students')
    if len(elem_list) != expected_rows:
        print(f'\n\n\n## Error for lista studenti. Trovati {len(elem_list)}, attesi {expected_rows}')
        
for i, el in enumerate(elem_list):
    grade=el['td'][8]['#text']
    lode = True if 'L' in grade else False
    grade_evaluated=int(grade.replace('L', ''))
    matricola=int(el['td'][1]['#text'])
    name=el['td'][0]['#text']
    grade_expected=min(df_studenti[df_studenti['Matricola'] == matricola]['Voto Finale'].values[0], 110)
    lode_expected = True if df_studenti[df_studenti['Matricola'] == matricola]['Lode'].values[0] != '' else False
    if grade_evaluated != grade_expected or lode != lode_expected:
        print(f'\n## Expected Votazione Finale mismatch: {name} ({i+1})')
        
print('\nVotazione Finale: Check Done')

Uploading student 9/9
Votazione Finale: Check Done
